In [31]:
from dune_client.client import DuneClient
dune = DuneClient("rVRuwpjjYaD1yXJAVH3dVZufZcESZK4E")
query_result = dune.get_latest_result(4818507)





In [33]:
import pandas as pd
# 提取 rows 数据
rows = query_result.result.rows

# 创建 DataFrame
df = pd.DataFrame(rows)

# 如果需要，将 transaction_date 列转换为 datetime 类型
df['transaction_day'] = pd.to_datetime(df['transaction_day'])

# 查看 DataFrame
df

,cumulative_unique_addresses,transaction_day,unique_addresses
0,1,2019-06-25,1
1,4,2019-06-27,3
2,38,2019-06-28,34
3,344,2019-06-29,306
4,347,2019-07-02,3
...,...,...,...
2079,57485457,2025-03-07,261986
2080,57711812,2025-03-08,226355
2081,57979394,2025-03-09,267582
2082,58306210,2025-03-10,326816


In [34]:
df.to_csv('kaia.csv', index=False)


In [26]:
import ccxt
import pandas as pd
import datetime

def get_kaia_historical_prices():
    # 初始化 Gate.io 交易所
    exchange = ccxt.gateio()

    # 设置交易对
    symbol = 'KAIA/USDT'  # 假设以 USDT 计价

    # 设置时间范围
    days_to_fetch = 500  # 总共获取 90 天的数据
    timeframe = '1d'  # 1天的时间框架
    all_data = []
    since = exchange.parse8601((datetime.datetime.now() - datetime.timedelta(days=days_to_fetch)).isoformat())

    # 分批获取数据
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
        if not ohlcv:
            break  # 如果没有数据则退出

        all_data.extend(ohlcv)  # 累积数据

        # 更新 since 为最后一个数据的时间戳 + 1 毫秒
        since = ohlcv[-1][0] + 1

    # 将结果转换为 DataFrame
    df = pd.DataFrame(all_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')  # 转换时间戳为日期
    df = df[['date', 'open', 'high', 'low', 'close', 'volume']]  # 重新排列列

    # 去除重复日期
    df = df.drop_duplicates(subset='date')

    return df  # 返回 DataFrame

# 调用函数并输出 DataFrame
df = get_kaia_historical_prices()
print(df)

          date     open     high      low    close        volume
0   2023-11-12  0.15000  0.16790  0.14350  0.16630  7.365046e+05
1   2023-11-13  0.16660  0.19460  0.16220  0.16240  2.015526e+06
2   2023-11-14  0.16210  0.16560  0.15260  0.16040  8.319437e+05
3   2023-11-15  0.16050  0.17390  0.15900  0.17350  6.130684e+05
4   2023-11-16  0.17350  0.17940  0.16670  0.17170  5.872530e+05
..         ...      ...      ...      ...      ...           ...
495 2025-03-21  0.10923  0.10982  0.10531  0.10608  2.344321e+06
496 2025-03-22  0.10626  0.10884  0.10626  0.10811  2.550505e+06
497 2025-03-23  0.10812  0.11222  0.10647  0.10732  2.233110e+06
498 2025-03-24  0.10729  0.11100  0.10609  0.10991  2.590372e+06
499 2025-03-25  0.10992  0.10993  0.10738  0.10824  1.838203e+06

[500 rows x 6 columns]


In [27]:
df.to_csv("kaia_price.csv")